# API version

## Intro

In [1]:
import hmac
import time
import hashlib
import requests
from urllib.parse import urlencode

from pandas import json_normalize

In [2]:
#! input your apikey and secret key

KEY = ''
SECRET = ''

# for spot market
# BASE_URL = 'https://api.binance.com' # production base url

# for future market
BASE_URL = 'https://fapi.binance.com' # production base url

In [3]:

''' ======  begin of functions, you don't need to touch ====== '''
def hashing(query_string):
    return hmac.new(SECRET.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def get_timestamp():
    return int(time.time() * 1000)


def dispatch_request(http_method):
    session = requests.Session()
    session.headers.update({
        'Content-Type': 'application/json;charset=utf-8',
        'X-MBX-APIKEY': KEY
    })
    return {
        'GET': session.get,
        'DELETE': session.delete,
        'PUT': session.put,
        'POST': session.post,
    }.get(http_method, 'GET')

# used for sending request requires the signature
def send_signed_request(http_method, url_path, payload={}):
    query_string = urlencode(payload, True)
    
    if query_string:
        query_string = "{}&timestamp={}".format(query_string, get_timestamp())
    else:
        query_string = 'timestamp={}'.format(get_timestamp())

    url = BASE_URL + url_path + '?' + query_string + '&signature=' + hashing(query_string)
    # print("{} {}".format(http_method, url))
    params = {'url': url, 'params': {}}
    response = dispatch_request(http_method)(**params)
    return response.json()

# used for sending public data request
def send_public_request(url_path, payload={}):
    query_string = urlencode(payload, True)
    url = BASE_URL + url_path
    if query_string:
        url = url + '?' + query_string
    # print("{}".format(url))
    response = dispatch_request('GET')(url=url)
    return response.json()

''' ======  end of functions ====== '''

' ======  end of functions ====== '

## Obtain the account info

In [ ]:
# "asset": "USDT",            // asset name
# "walletBalance": "23.72469206",      // wallet balance
# "unrealizedProfit": "0.00000000",    // unrealized profit
# "marginBalance": "23.72469206",      // margin balance
# "maintMargin": "0.00000000",        // maintenance margin required
# "initialMargin": "0.00000000",    // total initial margin required with current mark price 
# "positionInitialMargin": "0.00000000",    //initial margin required for positions with current mark price
# "openOrderInitialMargin": "0.00000000",   // initial margin required for open orders with current mark price
# "crossWalletBalance": "23.72469206",      // crossed wallet balance
# "crossUnPnl": "0.00000000"       // unrealized profit of crossed positions
# "availableBalance": "23.72469206",       // available balance
# "maxWithdrawAmount": "23.72469206",     // maximum amount for transfer out
# "marginAvailable": true,    // whether the asset can be used as margin in Multi-Assets mode
# "updateTime": 1625474304765 // last update time 

In [ ]:
# "symbol": "BTCUSDT",    // symbol name
# "initialMargin": "0",   // initial margin required with current mark price 
# "maintMargin": "0",     // maintenance margin required
# "unrealizedProfit": "0.00000000",  // unrealized profit
# "positionInitialMargin": "0",      // initial margin required for positions with current mark price
# "openOrderInitialMargin": "0",     // initial margin required for open orders with current mark price
# "leverage": "100",      // current initial leverage
# "isolated": true,       // if the position is isolated
# "entryPrice": "0.00000",    // average entry price
# "maxNotional": "250000",    // maximum available notional with current leverage
# "bidNotional": "0",  // bids notional, ignore
# "askNotional": "0",  // ask norional, ignore
# "positionSide": "BOTH",     // position side
# "positionAmt": "0",         // position amount
# "updateTime": 0           // last update time

In [16]:
# ## USER_DATA endpoints, call send_signed_request #####

response = send_signed_request('GET', '/fapi/v2/account')

account_assets = json_normalize(response['assets'])
account_positions = json_normalize(response['positions'])

account_positions = account_positions.set_index('symbol')
account_assets = account_assets.set_index('asset')

target_symbol = ['BTCUSDT','ETHUSDT']
asset = ['USDT']

In [17]:
account_assets.loc[asset]['marginBalance']

,walletBalance,unrealizedProfit,marginBalance,maintMargin,initialMargin,positionInitialMargin,openOrderInitialMargin,maxWithdrawAmount,crossWalletBalance,crossUnPnl,availableBalance,marginAvailable,updateTime
asset,,,,,,,,,,,,,
USDT,184.61690735,16.00338000,200.62028735,0.50005728,62.50716000,62.50716000,0.00000000,138.11312735,184.61690735,16.00338000,138.11312735,True,1641513600240


In [13]:
account_assets.loc[asset]

,walletBalance,unrealizedProfit,marginBalance,maintMargin,initialMargin,positionInitialMargin,openOrderInitialMargin,maxWithdrawAmount,crossWalletBalance,crossUnPnl,availableBalance,marginAvailable,updateTime
asset,,,,,,,,,,,,,
USDT,184.61690735,16.00019077,200.61709812,0.50007003,125.01750922,125.01750922,0.00000000,75.59958890,184.61690735,16.00019077,75.59958890,True,1641513600240


In [14]:
account_positions.loc[target_symbol]

,initialMargin,maintMargin,unrealizedProfit,positionInitialMargin,openOrderInitialMargin,leverage,isolated,entryPrice,maxNotional,positionSide,positionAmt,notional,isolatedWallet,updateTime,bidNotional,askNotional
symbol,,,,,,,,,,,,,,,,
BTCUSDT,125.01750922,0.50007003,16.00019077,125.01750922,0,1,False,47005.9,1.0E9,BOTH,-0.003,-125.01750922,0,1641178069794,0,0
ETHUSDT,0,0,0.00000000,0,0,1,False,0.0,5.0E8,BOTH,0.000,0,0,1641420260618,0,0


## Change Position Mode

In [25]:
params = {
    'dualSidePosition': 'false',
}

# "true": Hedge Mode; "false": One-way Mode
response = send_signed_request('POST', '/fapi/v1/positionSide/dual',params)
print(response)

POST https://fapi.binance.com/fapi/v1/positionSide/dual?dualSidePosition=false&timestamp=1641097285261&signature=c7fed7d16a7501cd08d369cb4486b4ecbc378a1b290e75997f631da2310579d3
{'code': -4059, 'msg': 'No need to change position side.'}


## Make order

### LIMIT

In [5]:
params = {
    'symbol': 'BTCUSDT',
    'side': 'SELL',
    'type': 'LIMIT',
    'timeInForce': 'GTC',
    'quantity': 0.001,
    'price': 45900
}

response = send_signed_request('POST', '/fapi/v1/order',params)
print(response)


# params = {
#     'symbol': 'ETHUSDT',
#     'side': 'SELL',
#     'type': 'LIMIT',
#     'timeInForce': 'GTC',
#     'quantity': 0.01,
#     'price': 3600
# }

# response = send_signed_request('POST', '/fapi/v1/order',params)
# print(response)

POST https://fapi.binance.com/fapi/v1/order?symbol=BTCUSDT&side=SELL&type=LIMIT&timeInForce=GTC&quantity=0.002&price=45900&timestamp=1640983483401&signature=f660cc095b4b080f569a771b4d598f9f83a2690fd9cc37a744b283612f7f9195
{'orderId': 39606056431, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'wWtnnmizY3xq2Cji8oaidM', 'price': '45900', 'avgPrice': '0.00000', 'origQty': '0.002', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'updateTime': 1640983483716}


### MARKET

In [8]:
symbol = 'BTCUSDT'
quantity = 0.003

params = {
    'symbol': symbol,
    'side': 'SELL',
    'type': 'MARKET',
    'quantity': quantity,
}

response = send_signed_request('POST', '/fapi/v1/order',params)
print(response)

{'code': -2019, 'msg': 'Margin is insufficient.'}


## Cancel Order

In [15]:
params = {
    'symbol': 'ETHUSDT',
    'orderId': '8389765513381323668',
}

response = send_signed_request('DELETE', '/fapi/v1/order',params)
print(response)

{'orderId': 8389765513381323668, 'symbol': 'ETHUSDT', 'status': 'CANCELED', 'clientOrderId': 'sMgn1u51E8XWULWEaAXmcR', 'price': '3600', 'avgPrice': '0.00000', 'origQty': '0.010', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'updateTime': 1641420040158}


## Query all Order (include the history order)

In [16]:
params = {
    'symbol': 'ETHUSDT',
}
response = send_signed_request('GET', '/fapi/v1/allOrders',params)
# print(response)
json_normalize(response)

,orderId,symbol,status,clientOrderId,price,avgPrice,origQty,executedQty,cumQuote,timeInForce,...,reduceOnly,closePosition,side,positionSide,stopPrice,workingType,priceProtect,origType,time,updateTime
0,8389765513191238801,ETHUSDT,FILLED,IaecjopRF8Qm6iwQ916hVs,3760,3749.21000,0.010,0.010,37.49210,GTC,...,False,False,BUY,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641097378388,1641097378388
1,8389765513192675471,ETHUSDT,CANCELED,7pUoxQue8zz0WqgSU3IqM5,3730,0.00000,0.010,0,0,GTC,...,False,False,SELL,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641100349990,1641100537658
2,8389765513192792805,ETHUSDT,FILLED,zb0BWGDa0g1CTYEBYJiHtf,3710,3722.99000,0.010,0.010,37.22990,GTC,...,False,False,SELL,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641100592486,1641100592486
3,8389765513217397663,ETHUSDT,FILLED,eCUzc5AMzeFK8kec96LwCw,3600,3600.00000,0.010,0.010,36,GTC,...,False,False,BUY,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641146007972,1641413292327
4,8389765513381323668,ETHUSDT,CANCELED,sMgn1u51E8XWULWEaAXmcR,3600,0.00000,0.010,0,0,GTC,...,False,False,SELL,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641419993000,1641420040158


## Query order

In [89]:
response['status']

'FILLED'

In [87]:
params = {
    'symbol': 'ETHUSDT',
    'orderId': 8389765513192792805,
    # 'origClientOrderId':"zb0BWGDa0g1CTYEBYJiHtf"
}
response = send_signed_request('GET', '/fapi/v1/order',params)
response

GET https://fapi.binance.com/fapi/v1/order?symbol=ETHUSDT&orderId=8389765513192792805&timestamp=1641146912013&signature=a4681278e9cce9a258e01459a3ea13f23599ff9c4164662702572e5768966f65


{'orderId': 8389765513192792805,
 'symbol': 'ETHUSDT',
 'status': 'FILLED',
 'clientOrderId': 'zb0BWGDa0g1CTYEBYJiHtf',
 'price': '3710',
 'avgPrice': '3722.99000',
 'origQty': '0.010',
 'executedQty': '0.010',
 'cumQuote': '37.22990',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'SELL',
 'positionSide': 'BOTH',
 'stopPrice': '0',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'LIMIT',
 'time': 1641100592486,
 'updateTime': 1641100592486}

## Query current open order

In [84]:
params = {
    'symbol': 'ETHUSDT',
    # 'orderId': 8389765513217397663,
    'origClientOrderId':"zb0BWGDa0g1CTYEBYJiHtf"
}
response = send_signed_request('GET', '/fapi/v1/openOrder',params)
response

GET https://fapi.binance.com/fapi/v1/openOrder?symbol=ETHUSDT&origClientOrderId=zb0BWGDa0g1CTYEBYJiHtf&timestamp=1641146824809&signature=448ff78cbc9c68b8c46a7419a81f1089db826ad8e6f2a6ed57a073b06aa9d5c7


{'code': -2013, 'msg': 'Order does not exist.'}

## Change the leverage

In [15]:
params = {
    'symbol': 'BTCUSDT',
    'leverage': 2,
    # 'origClientOrderId':"wWtnnmizY3xq2Cji8oaidM"
}
response = send_signed_request('POST', '/fapi/v1/leverage',params)
print(response)

{'symbol': 'BTCUSDT', 'leverage': 2, 'maxNotionalValue': '600000000'}


## commission

In [23]:
params = {
    'symbol': 'BTCUSDT',
    # 'origClientOrderId':"wWtnnmizY3xq2Cji8oaidM"
}
response = send_signed_request('GET', '/fapi/v1/commissionRate',params)
print(response)

GET https://fapi.binance.com/fapi/v1/commissionRate?symbol=BTCUSDT&timestamp=1641096886661&signature=47013642a44b9f9eb13eb2829389feb882b7c06475a7e95543b3c6aa51fcf1ff
{'symbol': 'BTCUSDT', 'makerCommissionRate': '0.000200', 'takerCommissionRate': '0.000400'}


# functions

## Module: Account balance

In [ ]:
def account_P_L():
    # ## USER_DATA endpoints, call send_signed_request #####
    response = send_signed_request('GET', '/fapi/v2/account')
    account_assets = json_normalize(response['assets'])
    account_assets = account_assets.set_index('asset')
    asset = ['USDT']
    
    Balance = account_assets.loc[asset]['walletBalance'].values[0]
    UnrealizedProfit = account_assets.loc[asset]['unrealizedProfit'].values[0]
    
    return float(Balance),float(UnrealizedProfit)
    
account_P_L()

## Module: Current position

In [121]:
def find(lst, value):
    # find the corresponding symbol info from the respond dict
    for i, dic in enumerate(lst):
        if dic['symbol'] == value:
            return lst[i]
    raise ValueError('No such a symbol')


def position(pos_amt):
    # judge the current position
    if pos_amt > 0:
        return "BUY"
    elif pos_amt < 0:
        return "SELL"
    else:
        return "Empty"
    
    
def Holding_info(symbol):
    
    # obtain the current account info
    response = send_signed_request('GET', '/fapi/v2/account')
    account_positions = json_normalize(response['positions'])

    # append the current position to the symbol holding information
    # short position of "SELL", long position for "BUY","Empty"
    symbol_info = find(response['positions'],symbol)
    symbol_pos_amt = float(symbol_info['positionAmt'])
    symbol_info['position'] = position(symbol_pos_amt)
    
    return symbol_info


def Current_position(symbol):
    hold_pos = Holding_info(symbol)['position']
    hold_amt = Holding_info(symbol)['positionAmt']
    return hold_pos,hold_amt
    

In [122]:
Current_position('BTCUSDT')

('SELL', '-0.002')

## Module: Order status

In [5]:
def Order_status(symbol:str,orderId:int):
    
    params = {
    'symbol': symbol,
    'orderId': orderId,
    # 'origClientOrderId':"zb0BWGDa0g1CTYEBYJiHtf"
    }
    
    response = send_signed_request('GET', '/fapi/v1/order',params)
    return response['status']
    
symbol = 'ETHUSDT'
orderId = 8389765513192792805
Order_status(symbol,orderId)


def Get_lastest_orderId(symbol):
    params = {
    'symbol': symbol,
    }
    response = send_signed_request('GET', '/fapi/v1/allOrders',params)
    lastest_order_id = response[-1]['orderId']
    lastest_order_status = response[-1]['status']
    
    return response[-1]['status']
    
    

In [6]:
symbol = 'ETHUSDT'
Get_lastest_orderId(symbol)

'NEW'

In [107]:
params = {
'symbol': 'ETHUSDT',
}
response = send_signed_request('GET', '/fapi/v1/allOrders',params)
# print(response)
json_normalize(response)

,orderId,symbol,status,clientOrderId,price,avgPrice,origQty,executedQty,cumQuote,timeInForce,...,reduceOnly,closePosition,side,positionSide,stopPrice,workingType,priceProtect,origType,time,updateTime
0,8389765513191238801,ETHUSDT,FILLED,IaecjopRF8Qm6iwQ916hVs,3760,3749.21000,0.010,0.010,37.49210,GTC,...,False,False,BUY,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641097378388,1641097378388
1,8389765513192675471,ETHUSDT,CANCELED,7pUoxQue8zz0WqgSU3IqM5,3730,0.00000,0.010,0,0,GTC,...,False,False,SELL,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641100349990,1641100537658
2,8389765513192792805,ETHUSDT,FILLED,zb0BWGDa0g1CTYEBYJiHtf,3710,3722.99000,0.010,0.010,37.22990,GTC,...,False,False,SELL,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641100592486,1641100592486
3,8389765513217397663,ETHUSDT,NEW,eCUzc5AMzeFK8kec96LwCw,3600,0.00000,0.010,0,0,GTC,...,False,False,BUY,BOTH,0,CONTRACT_PRICE,False,LIMIT,1641146007972,1641146007972


In [110]:
response[-1]['status']

'NEW'

## Module: Data obtain and indicator

In [ ]:
import pandas as pd

In [104]:
def future_data(pair,interval,contractType = 'PERPETUAL'):
    
    url_path = '/fapi/v1/continuousKlines'
    params = {
    'pair': pair,
    'contractType': contractType,
    'interval': interval,
    'limit':1500
    }

    response = send_public_request(url_path, params)

    col = ['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades',
        'Taker buy volume','Taker buy quote asset volume','Ignore']
    data = pd.DataFrame(response,columns=col)

    # adjust the datetime type
    data['Open time'] = pd.to_datetime(data['Open time'],unit='ms',utc=True)
    data['Close time'] = pd.to_datetime(data['Close time'],unit='ms',utc=True)

    # transfer the object to float
    data.iloc[:,1:6] = data.iloc[:,1:6].astype(float)
    
    return data


In [107]:
symbol = 'ETHUSDT'
interval = '8h'
data = future_data(symbol,interval)
data.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy volume,Taker buy quote asset volume,Ignore
0,2020-08-21 00:00:00+00:00,416.30,418.95,409.80,413.12,357477.447,2020-08-21 07:59:59.999000+00:00,148007543.83317,143706,158462.877,65628427.46620,0
1,2020-08-21 08:00:00+00:00,413.12,413.12,399.81,404.49,651229.917,2020-08-21 15:59:59.999000+00:00,264522052.31870,209160,292079.290,118644542.47656,0
2,2020-08-21 16:00:00+00:00,404.50,407.19,385.09,388.25,861459.161,2020-08-21 23:59:59.999000+00:00,340885436.08925,283731,378744.003,149917811.50611,0
3,2020-08-22 00:00:00+00:00,388.27,392.55,380.00,391.29,550641.210,2020-08-22 07:59:59.999000+00:00,212300747.57440,199969,263127.624,101514671.57880,0
4,2020-08-22 08:00:00+00:00,391.29,395.24,388.47,390.61,326870.272,2020-08-22 15:59:59.999000+00:00,128014028.09528,153323,151909.891,59513622.87287,0


In [74]:
import talib as ta

In [108]:
def ema_indicator(data):
    
    close = data['Close']
    ema_a = ta.EMA(close,timeperiod=36).tolist()
    ema_b = ta.EMA(close,timeperiod=60).tolist()

    return ema_a,ema_b
    

In [110]:
ema_a,ema_b = ema_indicator(data)

## Module: Trading Operation

In [9]:
def Order_BUY_Market(symbol,quantity):
    
    params = {
        'symbol': symbol,
        'side': 'BUY',
        'type': 'MARKET',
        'quantity': quantity,
    }
    response = send_signed_request('POST', '/fapi/v1/order',params)
    print(response)

def Order_SELL_Market(symbol,quantity = 0.002):
    
    params = {
        'symbol': symbol,
        'side': 'SELL',
        'type': 'MARKET',
        'quantity': quantity,
    }

    response = send_signed_request('POST', '/fapi/v1/order',params)
    print(response)

In [10]:
symbol = 'BTCUSDT'
quantity = 0.002

Order_SELL_Market(symbol,quantity)